In [ ]:
import os
print(os.getcwd())

import sys

from pymatgen.analysis.chemenv.coordination_environments.coordination_geometry_finder import LocalGeometryFinder

from pymatgen.analysis.chemenv.coordination_environments.chemenv_strategies import SimplestChemenvStrategy, MultiWeightsChemenvStrategy
from pymatgen.analysis.chemenv.coordination_environments.structure_environments import LightStructureEnvironments

import numpy as np

from pymatgen.core.structure import Structure

from pymatgen.io.ase import AseAtomsAdaptor

from ase.db import connect
from time import time

In [ ]:
import ase

ase

In [ ]:
t0 = time()

In [ ]:
db = connect('out_data/FinalStructures_1.db')

In [ ]:
for row in db.select():
    print('-------------------')
    print(row.id)

    atoms = row.toatoms()
    Ir_indices = [i for i, s in enumerate(atoms.get_chemical_symbols())
                  if s == 'Ir']

    struct = AseAtomsAdaptor.get_structure(atoms)


    lgf = LocalGeometryFinder()
    lgf.setup_structure(structure=struct)

    se = lgf.compute_structure_environments(
        maximum_distance_factor=1.41,
        only_cations=False)


    strategy = MultiWeightsChemenvStrategy.stats_article_weights_parameters()

    lse = LightStructureEnvironments.from_structure_environments(
        strategy=strategy, structure_environments=se)

    isite = 0

    cor_env = []
    for isite in Ir_indices:
        c_env = lse.coordination_environments[isite]
        if len(c_env) == 0:
            continue
        cor_env += [c_env[0]['ce_symbol']]

    if len(cor_env) == 0:
        continue
    uniques, counts = np.unique(cor_env, return_counts=True)

    if len(uniques) > 1:
        coor = 'mixed'

        coor_l = [int(c.split(':')[-1]) for c in cor_env]

        mean_coor = np.mean(coor_l)
        #idx = np.argmax(counts)
        #coor = uniques[idx]
    else:
        coor = uniques[0]
        mean_coor = int(coor.split(':')[-1])

    print(coor, mean_coor)
    db.update(id=row.id, coor_env=coor, mean_coor=mean_coor)
    

In [ ]:
print("Running time (s)", time() - t0)

In [ ]:
# struct

# se = lgf.compute_structure_environments(
#     maximum_distance_factor=1.41,
#     only_cations=False)